In [29]:
import os
import pandas as pd
import shutil
import json
import datetime

This script is designed to rearrange the videos and metadata from the original formats to prepare them into the final format for sharing with the users. 

1) the folder with the raw files is analyzed to see the list of video and metadata files present, as well as the list of whitelisted MAC addresses
2) the script then iterates through the video files. First it extracts the timecodes, and then makes a new folder for the corresponding date if there isn't already one
3) For each BLEMAC address listed in the metadata for that video, the script then creates a folders for the BLEMAC address if there isn't already one, and then creates a video and a metadata folder within this subfolder
4) the video and corresponding metadata file are then copied into these new folders

From this point, the raw files have all been successfully organized. The next part of the script acts to convert this data structure into a json so it is easily navigable, and then from there further processing can be done using this new file structure.


In [30]:
# listdir that ignores hidden files
def listdir(path):
    return sorted((f for f in os.listdir(path) if not f.startswith(".")), key=str.lower)

In [37]:
def unixtimeToDate(unixtime):
    unixToDatetime = datetime.datetime.fromtimestamp(1661324766) # Unix Time
    return str(unixToDatetime)[:10]


In [38]:
##function to take raw videos and metadata and reformat the file structure into the media folder
def organizeFiles():
        
    #get list of videos and metadata files from the raw video folder
    rawVideoNames= listdir("../raw/videos")
    rawMetaNames= listdir("../raw/metadata")
    whitelistedMacs=pd.read_csv("../mac_address_whitelist.csv")
    
    
    # the script then iterates through the video files. First it extracts the timecodes, and then makes a new folder for the corresponding date if there isn't already one
    
    for name in rawVideoNames:
        splitName=name.split(".mp4")[0]
        unixTime=splitName ##update once time is in metadata
        date=unixtimeToDate(unixTime)
        folders=os.listdir("../media/")
        if date not in folders:
            os.mkdir("../media/"+date)
            
            
        #load the corresponding metadata
        f = open("../raw/metadata/"+name.split(".mp4")[0]+".json", "r")
        metaJson = json.loads(f.read())
        #open json with that title
        bleIds=metaJson["bleMACs"]
        print(bleIds)
        
        #For each BLEMAC address listed in the metadata for that video, the script then creates a folders for the BLEMAC address if there isn't already one, and then creates a video and a metadata folder within this subfolder
        for bleId in bleIds:
            if bleId.upper() in whitelistedMacs["Mac Whitelist"].tolist():
                path="../media/"+date+"/"+bleId.replace(":","")
                
                dateFolders=os.listdir("../media/"+date)

                if bleId.replace(":","") not in dateFolders:
                    os.mkdir(path)
                    os.mkdir(path+"/metadata")
                    os.mkdir(path+"/videos")
                    
                #the video and corresponding metadata file are then copied into these new folders

                shutil.copyfile("../raw/videos/"+splitName+".mp4", path+"/videos/"+splitName+".mp4")
                shutil.copyfile("../raw/metadata/"+splitName+".json", path+"/metadata/"+splitName+".json")

organizeFiles()

['cc:b9:08:d9:d6:56', 'b8:bc:5b:4b:79:4a', 'af:47:08:11:15:bf', '67:6e:b5:49:6a:00']
['af:47:08:11:15:bf', 'cc:b9:08:d9:d6:56', '40:f5:20:89:32:6e', '5f:c4:40:37:d2:2a']
['cc:b9:08:d9:d6:56', 'af:47:08:11:15:bf']
['3c:1c:68:5e:57:d4', 'cc:b9:08:d9:d6:56', 'f8:dc:cc:e7:9c:32']
['4f:66:12:f9:d6:eb', '3c:1c:68:5e:57:d4', 'cc:b9:08:d9:d6:56']
['f8:dc:cc:e7:9c:32']
[]


In [15]:
#create object representation of timeline with videos and timecodes
def generateSequenceObject(rfidTag):
    sequence=[]
    for videoFilename in listdir("../media/"+rfidTag+"/videos"):
        videoId=videoFilename.split(".MP4")[0]
        f = open("../media/"+rfidTag+"/metadata/"+videoId+".json")
        data = json.load(f)
        timecode=data["unixTime"]
        videoObj={
            "videoId":videoId,
            "timecode":timecode
        }
        sequence.append(videoObj)  
    return sequence

#run generateSequenceObject over all the rfidTags and collect them in a single object
def generateAllSequenceObjects():
    sequences={}
    rfidTags=listdir("../media/")
    for rfidTag in rfidTags:
        sequences[rfidTag]=generateSequenceObject(rfidTag)
    return sequences
        
sequences=generateAllSequenceObjects()
print(sequences)

{'A': [{'videoId': 'A-1', 'timecode': 1666207575}], 'B': [{'videoId': 'B-1', 'timecode': 1666207675}], 'C': [{'videoId': 'C-1', 'timecode': 1666207775}], 'D': [{'videoId': 'D-1', 'timecode': 1666207875}]}


In [16]:
#Use Davinci Resolve API to create timelines for each rfidTag

#Create a Resolve timeline with the start timecode matching the first clip start timecode.
#Add all the clips in the correct ascending timecode order.
#Export an EDL from the timeline.
#Edit the EDL in a text editor. You should be able to copy the source timecode column in and out timecode to the record column in and out timecode.
#Save the new EDL.
#Import the new EDL which should conform a new sequence with all the clips in their appropriate places.
#Source clip timecode should match the sequence timecode.